In [1]:
import numpy as np
import xarray as xr
import copy 

In [ ]:
#Use CDO for regridding data to MPI-GE grid
#cdo -remapbil,MPI_GE_grid.txt in_file_original.nc out_file_GEgrid.nc

In [2]:
def window_stack(a, stepsize, r_window):
    return np.hstack( a[i:1+i-r_window or None:stepsize] for i in range(0,r_window) )


In [3]:
def r_anomaly(in_data_ref1,in_data_ref2,in_data_ref3,in_data_ref4,in_data_target,r_window):
    r_window_data1 = window_stack(in_data_ref1,1,r_window) #get data for running windows, model 1
    r_window_data2 = window_stack(in_data_ref2,1,r_window) #get data for running windows, model 2
    r_window_data3 = window_stack(in_data_ref3,1,r_window) #get data for running windows, model 3
    r_window_data4 = window_stack(in_data_ref4,1,r_window) #get data for running windows, model 4
    r_mean = np.concatenate((r_window_data1,r_window_data2,r_window_data3,r_window_data4),axis=1).mean(axis=1) # concatenate models and average over ensemble dim for the each window
    
    #subtract mean to original data
    start = int((r_window-1)/2)
    in_data_target[:start]= in_data_target[:start]-r_mean[0] #first missing values
    in_data_target[-start:] = in_data_target[-start:]-r_mean[-1] #last missing values
    in_data_target[start:-start] = in_data_target[start:-start]-r_mean #middle values
    return(in_data_target)

In [4]:
stepsize = 1
window = 15
start = int((window-1)/2)

# Load EOBS

In [42]:
# Europe domain
lon_min = -10
lon_max = 30
lat_min = 35 #36.5 if we want to compute trend: northern african part is mixed with nan and not nans..
lat_max = 70

In [43]:
path = '/work/uo1075/u241308/reanalysis/eobs/eobs_hw_projections/'
file = 'tasmax_1950_2022_GEgrid2.nc'

data_eobs = xr.open_dataset(path+file)
data_eobs = data_eobs.tx

#select area
data_eobs = data_eobs[:,(data_eobs.lat>=lat_min)&(data_eobs.lat<=lat_max),(data_eobs.lon>=lon_min)&(data_eobs.lon<=lon_max)]

#Get rid off 29th of februarys
data_eobs = data_eobs.sel(time=~((data_eobs.time.dt.month == 2) & (data_eobs.time.dt.day == 29)))

In [44]:
min_year_eobs = np.min(data_eobs.time.dt.year)
max_year_eobs = np.max(data_eobs.time.dt.year)

# MPI-GE

In [45]:
# Historical
run = ['HIST']
for n_run in run:
    # ------------------ Load first set -----------------------
    path = '/work/uo1075/u241308/grand_ensemble_cmip6/t2max/'
    file = 'nGE_Tmax_'+n_run+'_dm_land_europe.nc'
    
    with xr.open_dataset(path+file) as data1:
        data1 = data1.tasmax

        #Get rid off 29th of februarys
        data1 = data1.sel(time=~((data1.time.dt.month == 2) & (data1.time.dt.day == 29)))
        
    # ------------------ Load second set -----------------------
    path = '/work/uo1075/u241308/grand_ensemble_cmip6/t2max/ens31_50/'
    file = 'nGE_Tmax_'+n_run+'_dm_ens_31-50_land_europe.nc'
    
    with xr.open_dataset(path+file) as data2:
        data2 = data2.tasmax

        #Get rid off 29th of februarys
        data2 = data2.sel(time=~((data2.time.dt.month == 2) & (data2.time.dt.day == 29)))
        
    #concatenate over ensemble dimension
    data = np.concatenate((data1.values,data2.values),axis=1)

    #select EOBS years
    data_hist = data[data1.time.dt.year>=min_year_eobs]

In [46]:
# Historical
run = ['ssp245']
for n_run in run:
    # ------------------ Load first set -----------------------
    path = '/work/uo1075/u241308/grand_ensemble_cmip6/t2max/'
    file = 'nGE_Tmax_'+n_run+'_dm_land_europe.nc'
    
    with xr.open_dataset(path+file) as data1:
        data1 = data1.tasmax

        #Get rid off 29th of februarys
        data1 = data1.sel(time=~((data1.time.dt.month == 2) & (data1.time.dt.day == 29)))
        
    # ------------------ Load second set -----------------------
    path = '/work/uo1075/u241308/grand_ensemble_cmip6/t2max/ens31_50/'
    file = 'nGE_Tmax_'+n_run+'_dm_ens_31-50_land_europe.nc'
    
    with xr.open_dataset(path+file) as data2:
        data2 = data2.tasmax

        #Get rid off 29th of februarys
        data2 = data2.sel(time=~((data2.time.dt.month == 2) & (data2.time.dt.day == 29)))
        
        
    #concatenate over ensemble dimension
    data = np.concatenate((data1.values,data2.values),axis=1)

    #select EOBS years
    data_ssp = data[data1.time.dt.year<=max_year_eobs]

In [47]:
#concatenate 
data_model = np.concatenate((data_hist,data_ssp),axis=0)

#conver model data to celsius, as eobs in celsius
data_model = data_model -273.15

In [48]:
print(data_eobs.shape)
print(data_model.shape)

(26645, 19, 22)
(26645, 50, 19, 22)


In [49]:
#flatten lon/lat
data_MPI_flat = data_model.reshape(data_model.shape[0],data_model.shape[1],-1) 
data_eobs_flat = data_eobs.values.reshape(data_eobs.shape[0],-1) 

# Rest of SMILES

In [50]:
#Mask
path = '/work/uo1075/u241308/grand_ensemble_cmip6/t2max/'
file = 'nGE_Tmax_ssp245_dm_land_europe.nc'
mask = xr.open_dataset(path+file)
mask = mask.tasmax[0,0,:,:]

In [51]:
model_name = 'ACCESS-ESM1-5'
ens_number = '40'

# Historical
run = ['historical']
for index,n_run in enumerate(run):
    # ------------------ Load first set -----------------------
    path = '/work/uo1075/u241308/SMILE/'+model_name+'/tasmax/'
    file = model_name+'_Tmax_'+n_run+'_dm_ens_1-'+ens_number+'_GEgrid_europe.nc'
    
    with xr.open_dataset(path+file) as data1:
        data1 = data1.tasmax

        #Get rid off 29th of februarys
        data1 = data1.sel(time=~((data1.time.dt.month == 2) & (data1.time.dt.day == 29)))
        
    #select EOBS years
    data_hist = data1[data1.time.dt.year>=min_year_eobs]

    #latitudes are originaly inverted
    data_hist = data_hist.reindex(lat = data_hist.lat[::-1]) 
    #mask ocean data
    data_hist = np.where(mask==0,0,data_hist) 

# SSP245
run = ['ssp245']
for n_run in run:
    # ------------------ Load first set -----------------------
    path = '/work/uo1075/u241308/SMILE/'+model_name+'/tasmax/'
    file = model_name+'_Tmax_'+n_run+'_dm_ens_1-'+ens_number+'_GEgrid_europe.nc'
    with xr.open_dataset(path+file) as data1:
        data1 = data1.tasmax

        #Get rid off 29th of februarys
        data1 = data1.sel(time=~((data1.time.dt.month == 2) & (data1.time.dt.day == 29)))
        
    #select EOBS years
    data_ssp = data1[data1.time.dt.year<=max_year_eobs]

    #latitudes are originaly inverted
    data_ssp = data_ssp.reindex(lat = data_ssp.lat[::-1]) 
    #mask ocean data
    data_ssp = np.where(mask==0,0,data_ssp)

#concatenate 
data_model = np.concatenate((data_hist,data_ssp),axis=0)

#conver model data to celsius, as eobs in celsius
data_model = data_model -273.15

#flatten lon/lat
data_access_flat = data_model.reshape(data_model.shape[0],data_model.shape[1],-1)

In [52]:
print(data_hist.shape)
print(data_ssp.shape)
print(data_model.shape)

(23725, 40, 19, 22)
(2920, 40, 19, 22)
(26645, 40, 19, 22)


In [53]:
model_name = 'CanESM5'
ens_number = '25'

# Historical
run = ['historical']
for index,n_run in enumerate(run):
    # ------------------ Load first set -----------------------
    path = '/work/uo1075/u241308/SMILE/'+model_name+'/tasmax/'
    file = model_name+'_Tmax_'+n_run+'_dm_ens_1-'+ens_number+'_GEgrid_europe.nc'
    
    with xr.open_dataset(path+file) as data1:
        data1 = data1.tasmax

        #Get rid off 29th of februarys
        data1 = data1.sel(time=~((data1.time.dt.month == 2) & (data1.time.dt.day == 29)))
        
    #select EOBS years
    data_hist = data1[data1.time.dt.year>=min_year_eobs]

    #latitudes are originaly inverted
    data_hist = data_hist.reindex(lat = data_hist.lat[::-1]) 
    #mask ocean data
    data_hist = np.where(mask==0,0,data_hist) 

# SSP245
run = ['ssp245']
for n_run in run:
    # ------------------ Load first set -----------------------
    path = '/work/uo1075/u241308/SMILE/'+model_name+'/tasmax/'
    file = model_name+'_Tmax_'+n_run+'_dm_ens_1-'+ens_number+'_GEgrid_europe.nc'
    with xr.open_dataset(path+file) as data1:
        data1 = data1.tasmax

        #Get rid off 29th of februarys
        data1 = data1.sel(time=~((data1.time.dt.month == 2) & (data1.time.dt.day == 29)))
        
    #select EOBS years
    data_ssp = data1[data1.time.dt.year<=max_year_eobs]

    #latitudes are originaly inverted
    data_ssp = data_ssp.reindex(lat = data_ssp.lat[::-1]) 
    #mask ocean data
    data_ssp = np.where(mask==0,0,data_ssp)

#concatenate 
data_model = np.concatenate((data_hist,data_ssp),axis=0)

#conver model data to celsius, as eobs in celsius
data_model = data_model -273.15

#flatten lon/lat
data_can_flat = data_model.reshape(data_model.shape[0],data_model.shape[1],-1)

In [54]:
print(data_hist.shape)
print(data_ssp.shape)
print(data_model.shape)

(23725, 25, 19, 22)
(2920, 25, 19, 22)
(26645, 25, 19, 22)


In [55]:
model_name = 'MIROC6'
ens_number = '50'

# Historical
run = ['historical']
for index,n_run in enumerate(run):
    # ------------------ Load first set -----------------------
    path = '/work/uo1075/u241308/SMILE/'+model_name+'/tasmax/'
    file = model_name+'_Tmax_'+n_run+'_dm_ens_1-'+ens_number+'_GEgrid_europe.nc'
    
    with xr.open_dataset(path+file) as data1:
        data1 = data1.tasmax

        #Get rid off 29th of februarys
        data1 = data1.sel(time=~((data1.time.dt.month == 2) & (data1.time.dt.day == 29)))
        
    #select EOBS years
    data_hist = data1[data1.time.dt.year>=min_year_eobs]

    #mask ocean data
    data_hist = np.where(mask==0,0,data_hist) 

# SSP245
run = ['ssp245']
for n_run in run:
    # ------------------ Load first set -----------------------
    path = '/work/uo1075/u241308/SMILE/'+model_name+'/tasmax/'
    file = model_name+'_Tmax_'+n_run+'_dm_ens_1-'+ens_number+'_GEgrid_europe.nc'
    with xr.open_dataset(path+file) as data1:
        data1 = data1.tasmax

        #Get rid off 29th of februarys
        data1 = data1.sel(time=~((data1.time.dt.month == 2) & (data1.time.dt.day == 29)))
        
    #select EOBS years
    data_ssp = data1[data1.time.dt.year<=max_year_eobs]

    #mask ocean data
    data_ssp = np.where(mask==0,0,data_ssp)

#concatenate 
data_model = np.concatenate((data_hist,data_ssp),axis=0)

#conver model data to celsius, as eobs in celsius
data_model = data_model -273.15

#flatten lon/lat
data_miroc_flat = data_model.reshape(data_model.shape[0],data_model.shape[1],-1)

# Compute anomalies

In [56]:
#calculate running window ensemble mean and anomalies
r_anomaly_output = copy.deepcopy(data_eobs_flat)
for x in range(data_MPI_flat.shape[2]):
    in_data_mpi = data_MPI_flat[:,:,x]
    in_data_access = data_access_flat[:,:,x]
    in_data_can = data_can_flat[:,:,x]
    in_data_miroc = data_miroc_flat[:,:,x]
    in_data_eobs = data_eobs_flat[:,x]
    r_anomaly_output[:,x] = r_anomaly(in_data_mpi,in_data_access,in_data_can,in_data_miroc,in_data_eobs,window)
    #print(x)

#reshape to orignal dimensions       
r_anomaly_output = r_anomaly_output.reshape(data_eobs.shape[0],data_eobs.shape[1],data_eobs.shape[2])

/tmp/ipykernel_524267/2633202034.py:2: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.hstack( a[i:1+i-r_window or None:stepsize] for i in range(0,r_window) )


In [57]:
# ------------------ save data -----------------------
path = '/work/uo1075/u241308/reanalysis/eobs/eobs_hw_projections/'
file = 'tasmax_1950_2022_GEgrid2_anomaly_detrended_multi_model.nc'
data_eobs[:,:,:] = r_anomaly_output[:,:,:]
data_eobs.to_netcdf(path+file)

print('finished')

finished
